In [ ]:
#there are two scripts iro_iuc.py and blaCARB.py which take kleborate's Carbapenemase genes and IUC/IRO/RMP genes and search them against
#all assemblies. The results, iuciro.tsv and and carbGenes.tsv contain all the hits.
#They are cross referenced against the results of the plasmidfinder run in PlasmidFinderHits which are one per assembly

In [17]:
from os import listdir
from os.path import isfile, join, splitext
import numpy as np
from collections import Counter
import pandas as pd
import umap
import matplotlib.pyplot as plt
from Bio import SeqIO
import pickle as pkl

inputDir="/mnt/storage5/anton/KpReplicons/inputData/"
outputDir="/mnt/storage5/anton/KpReplicons/outputData/"
plasmidHits="/mnt/storage5/anton/KpReplicons/PlasmidFinderHits/"


In [18]:
# Load metadata on contigs: length, carbapenemase, sideriphores, assembly, plasmid replicons

with open("/mnt/storage5/anton/KpReplicons/ContigData.pkl", 'rb') as pkl_file:
    contigsData = pkl.load(pkl_file)

files = [f for f in listdir(plasmidHits) if isfile(join(plasmidHits, f))  and (splitext(f)[-1]==".fasta" or splitext(f)[-1]==".fna") and f[0:3]=="GCF"  ]

# Reading data from multiple files without known indices into dataframe is slow
# instead read it into dictionary and populate dataframe afterwards
contigPlasmids={} # key=contig ID, value=[plamid IDs]
contigAssembly={} # key=contig ID, value=assembly
for file in files:
    with open(plasmidHits+file) as plamidFile:
        for line in plamidFile:
            if line[0]==">":
                values=line.split(":")
                values[4]=values[4].split(" ")[0]
                if values[4] not in contigPlasmids:
                    contigPlasmids[values[4]]=set()
                contigPlasmids[values[4]].add(values[0].replace(">",""))
                contigAssembly[values[4]]=file

contigsMetaData=pd.DataFrame(index=contigsData["contigToassembly"].keys(), columns=["Assembly"])
contigsMetaData.index.name="Contig"
contigsMetaData["Assembly"]=[contigsData["contigToassembly"][f] for f in contigsMetaData.index]
contigsMetaData["Length"]=[contigsData["contigLenghts"][f] for f in contigsMetaData.index]
contigsMetaData["Replicons"]=[contigPlasmids[f] if f in contigPlasmids else "" for f in contigsMetaData.index]
contigsMetaData["iro"]=False
contigsMetaData["iuc"]=False
contigsMetaData["Carbapenemase"]=""
contigsMetaData["CarbapenemaseLength"]=0
contigsMetaData["HasCarbapenemase"]=False

# iuciro and carbGenes.tsv are both generated using BLAST of assemblies versus Kleborate AMR/siderophore database
with open(f'{inputDir}iuciro.tsv') as hvFile:
    for line in hvFile:
        values=line.split("\t")        
        for siderophore in ["iuc","iro"]:
            if  line.find(siderophore)>-1 and values[0] in contigsMetaData.index:
                contigsMetaData.at[values[0], siderophore]=True

with open(f'{inputDir}carbGenes.tsv') as carbFile:
    for line in carbFile:
        values=line.split("\t")
        if float(values[6])==100 and values[0] in contigsMetaData.index:
            if int(values[2])-int(values[1])>contigsMetaData.at[values[0], "CarbapenemaseLength"]:
                contigsMetaData.at[values[0], "Carbapenemase"]=values[3] if contigsMetaData.at[values[0],"Carbapenemase"]=="" else contigsMetaData.at[values[0], "Carbapenemase"]+";"+values[3]
                contigsMetaData.at[values[0], "CarbapenemaseLength"]=int(values[2])-int(values[1])
        contigsMetaData.at[values[0], "HasCarbapenemase"]=True # see if there are some imperfect matches for carbapenemases

# # Drop the contigs without carbapenemases, siderophores or replicons as they are not relevant for subsequent analysis
# contigsMetaData=contigsMetaData.loc[ (contigsMetaData["iuc"]) | (contigsMetaData["iro"]) | (contigsMetaData["Carbapenemase"]!="") ]

# Add the cluster assignment data to the contigsMetaData
embeddingClustersDF=pd.read_csv(f'{outputDir}/embedding.tsv', sep="\t", index_col=0, header=0)
for column in ["Cluster","Country","ST","CollectionYear", "Species"]:
    contigsMetaData[column]=[embeddingClustersDF.at[f,column] if f in embeddingClustersDF.index else ""  for f in contigsMetaData["Assembly"]]

contigsMetaData["Cluster"].replace(np.nan,"",inplace=True)
contigsMetaData["isChr"]=contigsMetaData["Length"]>4500000

# Drop assemblies that do not have cluster data as they are not relevant for subsequent analysis
contigsMetaData=contigsMetaData.loc[ contigsMetaData["Cluster"]!="" ]

In [41]:
clustersDescription=pd.DataFrame(index=set(contigsMetaData["Cluster"]), 
    columns=["N", "Main STs", "Total STs", "Species","Collection Dates","Main Replicons","Main Countries","Siderophores","Carbapenemases",
    "Contig with CR and replicon","Contig with HV and replicon", "Contig with CR and HV",
    "Chr with CR and replicon","Chr with HV and replicon", "Chr with CR and HV"])

for cluster in set(contigsMetaData["Cluster"]):
    clustersDescription.at[cluster,"N"]=len( set( contigsMetaData.loc[ contigsMetaData["Cluster"]==cluster, "Assembly" ] ) )
    clustersDescription.at[cluster,"Total STs"]=len( set( contigsMetaData.loc[ contigsMetaData["Cluster"]==cluster, "ST" ] ) )
    clustersDescription.at[cluster,"Species"]=', '.join(set( contigsMetaData.loc[ contigsMetaData["Cluster"]==cluster, "Species" ] ) )
    collectionYears=set(contigsMetaData.loc[ (contigsMetaData["CollectionYear"].notnull()) & (contigsMetaData["Cluster"]==cluster)  , "CollectionYear" ])
    clustersDescription.at[cluster,"Collection Dates"]=str(int(min( collectionYears )))+"-"+str(int(max( collectionYears )))
    
    replicons=[k for f in contigsMetaData.loc[contigsMetaData["Cluster"]==cluster, "Replicons"] for k in f]
    sortedReplicons={k: v for k, v in sorted(Counter(replicons).items(), key=lambda item: item[1], reverse=True)}
    clustersDescription.at[cluster,"Main Replicons"]=", ".join( [key+"("+str(sortedReplicons[key])+")" for key in sortedReplicons if sortedReplicons[key]>5 ]  )

    #These two columns require a particular style of aggregation
    for target, source  in zip(["Main STs","Main Countries"],["ST","Country"]):
        stsWithDuplicates=contigsMetaData.loc[ (contigsMetaData[source].notnull()) & (contigsMetaData["Cluster"]==cluster)]
        assemblySTs={}
        for index in stsWithDuplicates.index:
            assemblySTs[stsWithDuplicates.at[index,"Assembly"]]=stsWithDuplicates.at[index,source]

        sortedSTs={k: v for k, v in sorted(Counter(assemblySTs.values()).items(), key=lambda item: item[1], reverse=True)}
        clustersDescription.at[cluster,target]=", ".join( [key+"("+str(sortedSTs[key])+")" for key in sortedSTs if sortedSTs[key]>5 ]  )
    
    # "Contig with CR and replicon","Contig with HV and replicon", "Contig with CR and HV"
    for length, prefix in zip([0,4500000],["Contig","Chr"]):
        clustersDescription.at[cluster,f'{prefix} with CR and replicon']=Counter( (contigsMetaData["Cluster"]==cluster) & (contigsMetaData["Length"]>length) &
                ( (~contigsMetaData["iuc"]) & (~contigsMetaData["iro"]) ) & 
                ( contigsMetaData["Carbapenemase"]!="")  & 
                ([len(f)>0 for f in contigsMetaData["Replicons"]]) )[True]
        clustersDescription.at[cluster,f'{prefix} with HV and replicon']=Counter( (contigsMetaData["Cluster"]==cluster) & (contigsMetaData["Length"]>length) &
                ( (contigsMetaData["iuc"]) | (contigsMetaData["iro"]) ) & 
                ( contigsMetaData["Carbapenemase"]=="")  & 
                ([len(f)>0 for f in contigsMetaData["Replicons"]]) )[True]
        clustersDescription.at[cluster,f'{prefix} with CR and HV']=Counter( (contigsMetaData["Cluster"]==cluster) & (contigsMetaData["Length"]>length) &
                ( (contigsMetaData["iuc"]) | (contigsMetaData["iro"]) ) & 
                ( contigsMetaData["Carbapenemase"]!="")  & 
                ([len(f)>0 for f in contigsMetaData["Replicons"]]) )[True]

    # Add the siderophores and Bla_Carb_acquired basedon embeddingClustersDF (i.e. kleborate output)
    aerobactinAlleles=[f'{key}({str(value)})' for key,value in Counter(embeddingClustersDF.loc[embeddingClustersDF["Cluster"]==cluster]["Aerobactin"]).items() if value>5 and key!="-"]
    salcmochelinAlleles=[f'{key}({str(value)})' for key,value in Counter(embeddingClustersDF.loc[embeddingClustersDF["Cluster"]==cluster]["Salmochelin"]).items() if value>5 and key!="-"]
    clustersDescription.at[cluster,"Siderophores"]=",".join(aerobactinAlleles+salcmochelinAlleles)
    carbapenemaseAlleles=[f'{key}({str(value)})' for key,value in Counter(embeddingClustersDF.loc[embeddingClustersDF["Cluster"]==cluster]["Bla_Carb_acquired"]).items() if value>5 and key!="-"]
    clustersDescription.at[cluster,"Carbapenemases"]=",".join(carbapenemaseAlleles)



clustersDescription.to_csv(f'{outputDir}/clustersDescription.tsv',sep="\t")

In [20]:
geneLocations=contigsMetaData.loc[ (contigsMetaData["iuc"]) | (contigsMetaData["iro"] ) | 
                ( contigsMetaData["Carbapenemase"]!="")  | 
                ([len(f)>0 for f in contigsMetaData["Replicons"]])]
geneLocations.index.name="Contig"
geneLocations.to_csv(f'{outputDir}geneLocations.tsv',sep="\t")